In [1]:
from pyMez import *
from pyMez.Code.Analysis.SParameter import *

Importing pyMez, this should take roughly 30 seconds
Importing Code.DataHandlers.GeneralModels
It took 1.068 s to import Code.DataHandlers.GeneralModels
Importing Code.DataHandlers.HTMLModels
It took 0.174 s to import Code.DataHandlers.HTMLModels
Importing Code.DataHandlers.NISTModels
It took 3.256 s to import Code.DataHandlers.NISTModels
Importing Code.DataHandlers.TouchstoneModels
It took 0.005 s to import Code.DataHandlers.TouchstoneModels
Importing Code.DataHandlers.XMLModels
It took 0.125 s to import Code.DataHandlers.XMLModels
Importing Code.DataHandlers.ZipModels
It took 0.011 s to import Code.DataHandlers.ZipModels
Importing Code.InstrumentControl.Experiments
It took 1.148 s to import Code.InstrumentControl.Experiments
Importing Code.InstrumentControl.Instruments
It took 0.026 s to import Code.InstrumentControl.Instruments
Importing Code.Utils.Names
It took 0.019 s to import Code.Utils.Names
It took 5.833 s to import all of the active modules


In [2]:
# Required to unwrap phase and fix 90 degree phase flip from fit 
def fix_phase(phase_list):
    last_phase=phase_list[0]
    new_phase_list=[last_phase]
    for phase in phase_list[1:]:
        phase_offset=phase-last_phase
        #print("Phase Offset is {0}".format(phase_offset))
        if 90<phase_offset<270:

            new_phase=phase-180
        elif -90>phase_offset>-270:
            new_phase=phase+180
        elif -270>phase_offset:
            new_phase=phase+360
        else:
            new_phase=phase
        #print(" New Phase is {0}, Old Phase is {1}".format(new_phase,phase))
        new_phase_list.append(new_phase)
        last_phase=new_phase
    return new_phase_list

In [6]:
fundamental_frequency=1*10**9
tone_spacings=[10*10**6,5*10**6,2.5*10**6,250*10**3,25*10**3]
number_tones=11
number_power_phases=1
for tone_spacing in tone_spacings:
    # Here we have averaged the phase sweep for anything that deals with power or scattering parameters
    top_directory=r"C:\Share\AWG_Phase_Calibration\Schroeder_Phase_20181022\Full_Calibration"
    data_directory=os.path.join(top_directory,r"11_Tones_{0:0.0f}KHz_Spacing".format(10**-3*tone_spacing))
    frequency_array=np.linspace(fundamental_frequency-(number_tones-1)/2*tone_spacing,
                                fundamental_frequency+(number_tones-1)/2*tone_spacing,number_tones)
    # Measurements with one value per frequency --------------------------------------------------------------
    # The error adaptor was calulated using the MUF with the simplest possible models
    error_adapter=SNP(os.path.join(data_directory,"Solution_0.s2p"))
    # The corrected gamma's have a _0 after their name
    power_meter_gamma=S1PV1(os.path.join(data_directory,"Power_Meter_Gamma_11_Tones_{0:0.0f}KHz_Spacing_0.s1p".format(10**-3*tone_spacing)))
    scope_gamma=S1PV1(os.path.join(data_directory,"Scope_Gamma_11_Tones_{0:0.0f}KHz_Spacing_0.s1p".format(10**-3*tone_spacing)))

    # Measurements with 10 values per frequency or one per phase ---------------------------------------------
    a1_raw_magnitude_name=os.path.join(data_directory,r"A1_Amplitude_11_Tones_{0:0.0f}KHz_Spacing.txt".format(10**-3*tone_spacing))
    power_meter_summary_name=os.path.join(data_directory,r"Power_11_Tones_{0:03.0f}KHz_Spacing_Summary_20181022_011.txt".format(10**3*tone_spacing))
    phase_calibration_summary_name=os.path.join(data_directory,r"Phase_11_Tones_{0:03.0f}KHz_Spacing_Summary_20181022_011.txt".format(10**3*tone_spacing))
    a1_raw_magnitude_schema=read_schema(change_extension(a1_raw_magnitude_name,new_extension="schema"))
    a1_raw_magnitude=AsciiDataTable(a1_raw_magnitude_name,**a1_raw_magnitude_schema)
    power_meter_summary_schema=read_schema(change_extension(power_meter_summary_name,new_extension="schema"))
    power_meter_summary=AsciiDataTable(power_meter_summary_name,**power_meter_summary_schema)
    phase_calibration_summary_schema=read_schema(change_extension(phase_calibration_summary_name,new_extension="schema"))
    phase_calibration_summary=AsciiDataTable(phase_calibration_summary_name,**phase_calibration_summary_schema)
    
    ## Average Power Meter Readings
    power_meter_readings=[]
    for tone in range(number_tones):
        mean_reading=np.mean(np.array(power_meter_summary["Power"][tone*number_power_phases:(tone+1)*number_power_phases]))
        power_meter_readings.append(mean_reading)
    # Now we calculate |CA1|
    # Calculate Zpm
    z_power_meter=[]
    for row in power_meter_gamma.sparameter_complex[:]:

        z=50.*abs((1+row[1])/(1-row[1]))
        z_power_meter.append(z)

    c_a1_list=[]
    for index,row in enumerate(power_meter_gamma.sparameter_complex[:]):
        frequency=row[0]
        gamma_pm=row[1]
        EA22=error_adapter.sparameter_complex[index][4]
        EA21=error_adapter.sparameter_complex[index][3]
        a1_raw=float(a1_raw_magnitude["Amplitude_A1"][index])
        pm_reading=power_meter_readings[index]
        z_pm=z_power_meter[index]
    #     print("{0} is {1}".format("z_power_meter[index]",z_power_meter[index]))
    #     print("{0} is {1}".format("gamma_pm",gamma_pm))
    #     print("{0} is {1}".format("EA22",EA22))
    #     print("{0} is {1}".format("EA21",EA21))
    #     print("{0} is {1}".format("a1_raw",a1_raw))
    #     print("{0} is {1}".format("power_meter_readings[index]",power_meter_readings[index]))
        c_a1=np.sqrt(pm_reading*z_pm)*abs(1-gamma_pm*EA22)/abs(a1_raw*EA21)
        c_a1_list.append(c_a1)

    calculated_scope_amplitudes=[]
    for index,row in enumerate(scope_gamma.sparameter_complex[:]):
        frequency=row[0]
        gamma_scope=row[1]
        EA22=error_adapter.sparameter_complex[index][4]
        EA21=error_adapter.sparameter_complex[index][3]
        a1_raw=float(a1_raw_magnitude["Amplitude_A1"][index])
        c_a1=c_a1_list[index]
    #     print("{0} is {1}".format("z_power_meter[index]",z_power_meter[index]))
    #     print("{0} is {1}".format("gamma_pm",gamma_pm))
    #     print("{0} is {1}".format("EA22",EA22))
    #     print("{0} is {1}".format("EA21",EA21))
    #     print("{0} is {1}".format("a1_raw",a1_raw))
    #     print("{0} is {1}".format("power_meter_readings[index]",power_meter_readings[index]))
        calculated_scope_amplitude=abs(c_a1*a1_raw*EA21)/abs(1-gamma_scope*EA22)
        calculated_scope_amplitudes.append(calculated_scope_amplitude)
    # Now we compare with the average of the scope readings
    scope_amplitudes=[]
    for tone in range(11):
        mean_reading=np.mean(abs(np.array(phase_calibration_summary["Scope_Amplitude"][tone*10:(tone+1)*10])))
        scope_amplitudes.append(mean_reading)
    plt.close()
    plt.plot(frequency_array,scope_amplitudes,"r^",label="Scope Measurement Amplitude")
    plt.plot(frequency_array,calculated_scope_amplitudes,"bs",label="Calculated Amplitude")
    plt.legend()
    plt.savefig(data_directory+"/Scope_Versus_Ascope.png")
    plt.close()
    plt.plot(frequency_array,np.array(scope_amplitudes)/np.array(calculated_scope_amplitudes),"rx",label="Scope Measurement Amplitude")
    plt.legend()
    plt.savefig(data_directory+"/h_scope.png")
    plt.close()
    h_scope=np.array(scope_amplitudes)/np.array(calculated_scope_amplitudes)
    plt.hist(h_scope)
    plt.title("Mean h:{0:3.3f}, Std h:{1:3.3f}".format(np.mean(h_scope),np.std(h_scope)))
    plt.savefig(data_directory+"/histogram_h_scope.png")
    # Now we find the phases of the scope and correct them for gamma
    scope_measured_phases=phase_calibration_summary["Scope_Phase"][:]
    scope_reference_phases=[]
    c_a1_phases=[]
    c_awg_phases=[]
    for tone in range(11):
        phase_row=[]
        calibration_phase_row=[]
        raw_row=[]
        awg_phase_row=[]
        for index,phase in enumerate(fix_phase(scope_measured_phases[tone*10:(tone+1)*10])):
            a_scope_measured=h_scope[index//10]*cmath.rect(abs(float(phase_calibration_summary["Scope_Amplitude"][index])),phase*np.pi/180.)
            gamma_scope=scope_gamma.sparameter_complex[index//10][1]
            angle_a_raw=float(fix_phase(phase_calibration_summary["Vna_Phase"])[index])
            amplitude_a_raw=float(a1_raw_magnitude["Amplitude_A1"][index//10])
            EA22=error_adapter.sparameter_complex[index//10][4]
            EA21=error_adapter.sparameter_complex[index//10][2]
            reference_phase=np.angle(a_scope_measured*(1-gamma_scope*EA22)/(amplitude_a_raw*EA21),deg=True)
            phase_row.append(reference_phase)
            raw_row.append(angle_a_raw)
            a_awg_amplitude=float(phase_calibration_summary["AWG_Set_Amplitude"][index])
            a_awg_phase=float(phase_calibration_summary["AWG_Set_Phase"][index])
            awg_phase_row.append(a_awg_phase)
            #alibration_phase=reference_phase-angle_a_raw
            #alibration_phase_row.append(calibration_phase)
    #         print("{0} is {1}".format("phase",phase))
    #         print("{0} is {1}".format("amplitude_a_raw",amplitude_a_raw))
    #         print("{0} is {1}".format("reference_phase",reference_phase))
    #         print("{0} is {1}".format("calibration_phase",calibration_phase))

        phase_row=np.array(fix_phase(phase_row))
        raw_row=np.array(fix_phase(raw_row))
        awg_phase_row=np.array(fix_phase(awg_phase_row))

        calibration_phase_row=phase_row-raw_row
        awg_calibratiton_phase_row=phase_row-awg_phase_row
        scope_reference_phases.append(np.mean(phase_row))
        c_a1_phases.append(np.mean(calibration_phase_row))
        c_awg_phases.append(np.mean(awg_calibratiton_phase_row))
    c_a1_phases=fix_phase(c_a1_phases)
    c_awg_phases=fix_phase(c_awg_phases)
    # Now we can save the calibration coefficients as a s1p.
    c_a1_complex=[]
    for index,phase in enumerate(c_a1_phases[:]):
        frequency=float(a1_raw_magnitude["Frequency"][index])
        ca1=cmath.rect(c_a1_list[index],phase*np.pi/180.)
        c_a1_complex.append([frequency,ca1])
    correction_coefficent=S1PV1(None,sparameter_complex=c_a1_complex)
    correction_coefficent.path=os.path.join(data_directory,"C_A1.s1p")
    correction_coefficent.save()
    c_awg_complex=[]
    for index,phase in enumerate(c_awg_phases[:]):
        frequency=float(a1_raw_magnitude["Frequency"][index])
        a1_raw=float(a1_raw_magnitude["Amplitude_A1"][index])
        mag_cawg=c_a1_list[index]*a1_raw/.5
        cawg=cmath.rect(mag_cawg,phase*np.pi/180.)
        c_awg_complex.append([frequency,cawg])
    awg_correction_coefficent=S1PV1(None,sparameter_complex=c_awg_complex)
    awg_correction_coefficent.path=os.path.join(data_directory,"C_AWG.s1p")
    awg_correction_coefficent.save()
    # Now we can calculate C_awg_scope
    c_awg_scope=[]
    for tone in range(number_tones):
        gamma_scope=scope_gamma.sparameter_complex[tone][1]
        EA22=error_adapter.sparameter_complex[tone][4]
        EA21=error_adapter.sparameter_complex[tone][3]
        h=h_scope[tone]
        cawg=c_awg_complex[tone][1]
        frequency=float(a1_raw_magnitude["Frequency"][tone])
        c=(1-gamma_scope*EA22)/(h*cawg*EA22)
        c_awg_scope.append([frequency,c])
    scope_awg_correction_coefficent=S1PV1(None,sparameter_complex=c_awg_scope)
    scope_awg_correction_coefficent.path=os.path.join(data_directory,"C_Awg_Scope.s1p")
    scope_awg_correction_coefficent.save()

In [ ]:
# Now lets compare all the values at 1GHz